In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [2]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            labels.append(label)
    return images, labels

In [3]:
aadhar_card_images, aadhar_card_labels = load_images_from_folder('C:/Users/Vivenns/OneDrive - Victorian Institute of Technology/Desktop/model/aadhar_card_detection/addhar_card', 1)
non_aadhar_card_images, non_aadhar_card_labels = load_images_from_folder('C:/Users/Vivenns/OneDrive - Victorian Institute of Technology/Desktop/model/aadhar_card_detection/non_addhar_card', 0)

In [4]:
all_images = aadhar_card_images + non_aadhar_card_images
all_labels = aadhar_card_labels + non_aadhar_card_labels

In [5]:
def preprocess_images(images):
    preprocessed_images = []
    for img in images:
        resized_img = cv2.resize(img, (224, 224))
        normalized_img = resized_img.astype('float32') / 255.0
        preprocessed_images.append(normalized_img)
    return np.array(preprocessed_images)

In [6]:
preprocessed_images = preprocess_images(all_images)
preprocessed_labels = np.array(all_labels)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_images, preprocessed_labels, test_size=0.2, random_state=42)


In [8]:
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: (21, 224, 224, 3)
Test data shape: (6, 224, 224, 3)


In [9]:
# X_train

In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [11]:
for layer in base_model.layers:
    layer.trainable = False

In [12]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [13]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5238 - loss: 0.7482 - val_accuracy: 0.1667 - val_loss: 16.3652
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5238 - loss: 8.3498 - val_accuracy: 0.1667 - val_loss: 1.5200
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.7619 - loss: 0.4577 - val_accuracy: 0.8333 - val_loss: 0.6945
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8571 - loss: 0.7078 - val_accuracy: 0.8333 - val_loss: 1.1535
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.6667 - loss: 1.5251 - val_accuracy: 0.8333 - val_loss: 0.6454
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8095 - loss: 0.3896 - val_accuracy: 1.0000 - val_loss: 0.0440
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0341 - val_accuracy: 1.0000 - val_loss: 9.8494e-04
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0090 - val_accuracy: 1.0000 - val_loss: 0.0086
Epoch 9/10


In [15]:
model.save('aadhar_card_model.h5')